# Skip Reindexing and Reuse Saved Embeddings


In this notebook, we will skip the embedding generation and re-indexing process by loading previously saved document embeddings and metadata from a JSON file. We will then use this data to directly query Elasticsearch and the RetrievalQA chain.
    

In [1]:
import json
import numpy as np
from elasticsearch import Elasticsearch
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.schema import Document
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
import os


# Load the saved embeddings and metadata
with open('document_embeddings.json', 'r') as f:
    saved_data = json.load(f)

# Extract texts, embeddings, and metadata
texts = [item['text'] for item in saved_data]
embeddings = [np.array(item['embedding']) for item in saved_data]
metadatas = [item['metadata'] for item in saved_data]

print("Loaded embeddings and metadata from JSON.")
    

Loaded embeddings and metadata from JSON.


In [2]:

# Initialize Elasticsearch connection
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])
index_name = 'fcra_chunks'

# Check if the index already exists
if es.indices.exists(index=index_name):
    print(f"Index '{index_name}' already exists in Elasticsearch. No re-indexing required.")
else:
    print(f"Index '{index_name}' does not exist. Re-indexing is required.")
    index_name = 'fcra_chunks'

    # Delete the index if it already exists (optional)
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)

    # Define the mapping
    mapping = {
        "mappings": {
            "properties": {
                "embedding": {
                    "type": "dense_vector",
                    "dims": 1536
                },
                "text": {
                    "type": "text"
                },
                "metadata": {
                    "type": "object",
                    "enabled": True
                }
            }
        }
    }

    # Create the index with the mapping
    es.indices.create(index=index_name, body=mapping)


Index 'fcra_chunks' does not exist. Re-indexing is required.


In [3]:
#  Load environment variables
load_dotenv()

# Initialize the embedding model with the API key
embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_28197/3451141637.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))


In [4]:
# Create a Custom Retriever Using Elasticsearch

from langchain.schema import Document
from langchain.schema import BaseRetriever
from typing import Any, List
import numpy as np
from pydantic import BaseModel

class ElasticSearchRetriever(BaseRetriever, BaseModel):
    es: Any
    index_name: str
    embedding_model: Any
    k: int = 5

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str) -> List[Document]:
        # Generate and normalize the query embedding
        query_embedding = self.embedding_model.embed_query(query)
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        # Build the script score query
        script_query = {
            "size": self.k,
            "query": {
                "script_score": {
                    "query": {"match_all": {}},
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                        "params": {
                            "query_vector": query_embedding.tolist()
                        }
                    }
                }
            },
            "_source": ["text", "metadata"]
        }

        # Execute the search
        response = self.es.search(index=self.index_name, body=script_query)

        # Convert hits to Documents
        docs = []
        for hit in response['hits']['hits']:
            doc = Document(
                page_content=hit['_source']['text'],
                metadata=hit['_source']['metadata']
            )
            docs.append(doc)
        return docs

# Initialize the Retriever
retriever = ElasticSearchRetriever(
    es=es,
    index_name=index_name,
    embedding_model=embedding_model,
    k=5  # Number of documents to retrieve
)

/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_28197/4229264796.py:9: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class ElasticSearchRetriever(BaseRetriever, BaseModel):


In [5]:

# Initialize the LLM (GPT)
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",  # You can change to map_reduce, etc.
    retriever=retriever
)

# Run a test query
query = "What are the key provisions of the FCRA?"
answer = qa_chain.run(query)
print("Answer:", answer)
    

/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_28197/3825728049.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))
/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_28197/3825728049.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


Answer:  The FCRA (Fair Credit Reporting Act) is a federal law that regulates the collection, dissemination, and use of consumer credit information. It is intended to protect consumers from inaccurate or unfair credit reporting practices.

Some key provisions of the FCRA include:
1. The requirement for consumer reporting agencies to ensure the accuracy of the information they collect and report.
2. The right for consumers to request and obtain a free credit report once every 12 months from each of the three major credit bureaus.
3. The requirement for companies to obtain a consumer's consent before obtaining their credit report.
4. The ability for consumers to dispute inaccurate information on their credit report and have it corrected or removed.
5. The requirement for companies to notify consumers if adverse actions are taken based on information in their credit report, such as denial of credit or employment.
6. The limitation on the length of time certain negative information can be 

In [6]:
queries = [
    "What are the permissible purposes for obtaining a consumer report?",
    "Explain the dispute process under the FCRA. Which module did you used to answer this question?",
    "What obligations do credit reporting agencies have according to the FCRA? Which module did you used to answer this question?"
]

for query in queries:
    answer = qa_chain.run(query)
    print(f"Query: {query}")
    print(f"Answer: {answer}")
    print("------")

Query: What are the permissible purposes for obtaining a consumer report?
Answer: 

Permissible purposes for obtaining a consumer report include employment purposes, credit applications, insurance underwriting, and rental or lease agreements. 
------
Query: Explain the dispute process under the FCRA. Which module did you used to answer this question?
Answer:  The Fair Credit Reporting Act (FCRA) provides consumers with the right to dispute and correct inaccurate information on their credit reports. This can be done by filing a dispute with the credit reporting agency or directly with the creditor. The agency or creditor must then investigate the dispute and either correct or delete the inaccurate information. To answer this question, I used my knowledge of financial regulations and the FCRA module.
------
Query: What obligations do credit reporting agencies have according to the FCRA? Which module did you used to answer this question?
Answer: 

Credit reporting agencies have obligation